## Model Development using pretrained deeplearning 
- load dataset
- split to training, validation and testing
- build pretrained model
- train model
- evaluate - save


In [8]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate, concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import tensorflow as tf


In [15]:
def create_mlp(x):
    x = Dense(256, activation="relu")(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(128, activation="relu")(x) 
    return x

def mode4text_feature(x):
    x = Dense(256, activation="relu")(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(128, activation="relu")(x) 
    return x

def pretrained_cnn_model(inputs:Input):
    mobilenetv2 = tf.keras.applications.MobileNetV2(
        input_tensor = inputs, 
        weights="imagenet", include_top=False, alpha=0.35) 
    x = mobilenetv2.get_layer('out_relu').output
    x = GlobalAveragePooling2D(name='gap')(x)
    x = Dense(128,activation='relu')(x)
    return x

In [16]:
def build_model(input1_shape, input2_shape,input3_shape):
    input1 = Input(shape=input1_shape, name="input_image")
    input2 = Input(shape=input2_shape, name="input_numerical") 
    input3 = Input(shape=input3_shape, name="input_text")  
    
    x_image = pretrained_cnn_model(input1)
    x_numerical = create_mlp(input2)
    x_text = mode4text_feature(input3)
    
    x = Concatenate(axis=0)([x_image,x_numerical,x_text])
    x = Dense(20, activation='relu')(x)
    output = Dense(1)(x)
    
    return tf.keras.Model([input1,input2,input3],output)

In [17]:
model = build_model(input1_shape=(256,256,3), input2_shape=(512,),input3_shape=(512,))
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])
model.summary() 

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 16  432         ['input_image[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 16  64          ['Conv1[0][0]']                  
                                )                                                           